# Demo: Building a Decision Tree Classification Model with IBM Db2

## Table of Contents:
* [1. Introduction](#Introduction)
* [2. Imports](#Imports)
* [3. Connect to DB](#Connect-to-DB)
* [4. Data Exploration](#EDA)
* [5. Splitting Data into Training, Validation, and Test Sets](#Split-Data)
* [6. Data Transformation](#Data-Transformation)
* [7. Model Training](#Model-Training)
* [8. Evaluate Unpruned Model on Training and Test Set](#Eval-UP)
* [9. Hyperparameter Tuning](#Hyperparam-Tuning)
* [10. Evaluate Pruned Model on Training and Test Set](#Eval-P-Test)
* [11. Deployed Model](#Deployment)
* [12. Conclusion](#Conclusion)
* [13. Cleaning up Model and Tables](#Cleaning-up)

## 1. Introduction <a class="anchor" id="Introduction"></a>


In this notebook, we demonstrate the use of Db2 Stored Procedures in building a Machine Learning Pipeline.


We will build a Decision Tree Classifier using the Titanic Dataset. This model will predict which passengers survived the Titanic shipwreck. The dataset contains 891 rows and 12 features (one of which is the target feature).

|Variable | Definition | Key|
| --- | --- |--- |
|survival | Survival | 0 = No, 1 = Yes|
|pclass | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd|
|sex | Sex|
|Age | Age in years|
|sibsp | # of siblings / spouses aboard the Titanic|
|parch | # of parents / children aboard the Titanic|
|ticket | Ticket number|
|fare | Passenger fare| 
|cabin | Cabin number|
|embarked | Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton|

## 2. Imports <a class="anchor" id="Imports"></a>

In [ ]:
# Database connectivity
import ibm_db
import ibm_db_dbi

# Pandas for loading values into memory for later visualization
import pandas as pd
from IPython.display import display
import numpy as np
import scipy.stats as ss
from itertools import combinations_with_replacement

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(12,8)})
%config InlineBackend.figure_format = 'retina'

## 3. Connect to DB<a class="anchor" id="Connect-to-DB"></a>

In [ ]:
# Connect to DB

schema = "CLASS"

conn_str = "DATABASE=______;" + \
           "HOSTNAME=______;" + \
           "PROTOCOL=______;"  + \
           "PORT=______;" + \
           "UID=______;" + \
           "PWD=_______;"


ibm_db_conn = ibm_db.connect(conn_str,"","")
conn = ibm_db_dbi.Connection(ibm_db_conn)
print('Connection to Db2 Instance Created!')
rc = ibm_db.close(ibm_db_conn)

### Custom functions

In [ ]:
# Function for connecting to a particular DB schema
def connect_to_schema(schema, conn_str):
    """Connect to a particular DB schema.
    
    Input:  schema - name of schema in Db2 to connect to
            conn_str - a Db2 connection string
    
    Output: none
    """
    ibm_db_conn = ibm_db.connect(conn_str,"","")
    conn = ibm_db_dbi.Connection(ibm_db_conn)

    sql = "set schema "+schema
    stmt = ibm_db.exec_immediate(ibm_db_conn, sql)
    
    return ibm_db_conn, conn

In [ ]:
# Function for plotting confusion matrix
def plot_conf_mtx(table_name):
    """Plot a confusion matrix similar in style to sklearn.
    
    Input:  table_name - Confusion matrix table generated by IDAX.CONFUSION_MATRIX
    
    Output: Confusion matrix plot
    """
    
    ibm_db_conn, conn = connect_to_schema(schema,conn_str)
    sql = "SELECT CNT FROM " + table_name

    arr = pd.read_sql(sql,conn).to_numpy()
    conf_matrix=np.vstack( (np.hstack((arr[0],arr[1])),np.hstack((arr[2],arr[3]) ) ))
    
    group_names = ['True Negatives','False Positives','False Negatives','True Positives']
    group_counts = ["{0:0.0f}".format(value) for value in conf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in conf_matrix.flatten()/np.sum(conf_matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    
    sns.heatmap(conf_matrix, annot=labels, fmt='', cmap='Blues')
    plt.xlabel('Predicted Value')
    plt.ylabel('Actual Value')
    rc = ibm_db.close(ibm_db_conn)
    print('Connection Closed:',rc)
    

In [ ]:
# Custom function for collecting statistics from confusion matrix
# Function for collecting statistics from confusion matrix
def get_conf_mtx_stats(matrixTable):
    """Get Classification Accuracy, Precision, Recall from a confusion matrix.
    
    Input:  matrixTable - Confusion matrix table generated by IDAX.CONFUSION_MATRIX
    
    Output: Classification Accuracy, Precision, Recall
    """
    
    ibm_db_conn, conn = connect_to_schema(schema,conn_str)

    sql = "CALL IDAX.CMATRIX_STATS('matrixTable="+matrixTable +"')"
    stmt = ibm_db.exec_immediate(ibm_db_conn, sql)
    
    
    df = pd.DataFrame(columns=['Class', 'Precision', 'Recall'])
    
    row = ibm_db.fetch_assoc(stmt)
    while row != False :
        print("Classification Accuracy: ","{:.1%}".format(row["ACC"]))
        row = ibm_db.fetch_assoc(stmt)

    stmt1 = ibm_db.next_result(stmt)
    while stmt1 != False:
        row = ibm_db.fetch_assoc(stmt1)
        while row != False :
            to_append=[row["CLASS"],"{:.1%}".format(row["PPV"]),"{:.1%}".format(row["TPR"])]
            df_length = len(df)
            df.loc[df_length] = to_append
            row = ibm_db.fetch_assoc(stmt1)
        stmt1 = ibm_db.next_result(stmt)
    display(df)
    rc = ibm_db.close(ibm_db_conn)
    print('Connection Closed:',rc) 

In [ ]:
# Function for plotting CDFs from runstats statistics
def plot_cdf_from_runstats_quartiles(col_name,quartiles_df,percentiles=True,norm_cdf=True):
    
    """Plot feature CDF from quartiles statistics from Db2 RUNSTATS.
    
    Input:  col_name - Name of the feature
            quartiles_df - DataFrame from RUNSTATS with Type = 'Q'
            percentiles=True - Plot 25th, 50th, 75th percentiles
            norm_cdf=True - Plot CDF of feature normally distributed over its range
    
    Output: Plots feature CDF
    """
    
    
    COL_HISTOGRAM = quartiles_df[(quartiles_df["COLNAME"] == col_name)].copy()
    COL_HISTOGRAM["VAL_PERCENT"]=COL_HISTOGRAM["VALCOUNT"]/891
    
    sns.lineplot(x='COLVALUE', y='VAL_PERCENT', data=COL_HISTOGRAM, label="CDF", lw=2, err_style=None)
    
    # Plot dashed lines indicating points of 25th, 50th, and 75th percentiles
    if percentiles:
        plt.hlines(y=0.25, xmin=0, xmax=COL_HISTOGRAM.iloc[(COL_HISTOGRAM['VAL_PERCENT']-0.25).abs().argsort()[:1]]['COLVALUE'], colors='r', linestyles='--',label='25th Percentile')
        plt.vlines(x=COL_HISTOGRAM.iloc[(COL_HISTOGRAM['VAL_PERCENT']-0.25).abs().argsort()[:1]]['COLVALUE'], ymin=0, ymax=0.25, colors='r', linestyles='--',label='_nolegend_')
    
        plt.hlines(y=0.50, xmin=0, xmax=COL_HISTOGRAM.iloc[(COL_HISTOGRAM['VAL_PERCENT']-0.50).abs().argsort()[:1]]['COLVALUE'], colors='g', linestyles='--',label='50th Percentile')
        plt.vlines(x=COL_HISTOGRAM.iloc[(COL_HISTOGRAM['VAL_PERCENT']-0.50).abs().argsort()[:1]]['COLVALUE'], ymin=0, ymax=0.50, colors='g', linestyles='--',label='_nolegend_')
    
        plt.hlines(y=0.75, xmin=0, xmax=COL_HISTOGRAM.iloc[(COL_HISTOGRAM['VAL_PERCENT']-0.75).abs().argsort()[:1]]['COLVALUE'], colors='k', linestyles='--',label='75th Percentile')
        plt.vlines(x=COL_HISTOGRAM.iloc[(COL_HISTOGRAM['VAL_PERCENT']-0.75).abs().argsort()[:1]]['COLVALUE'], ymin=0, ymax=0.75, colors='k', linestyles='--',label='_nolegend_')
    
    # Plot CDF of normal distribution where mean = RANGE/2, with same stddev as original distribution
    if norm_cdf:
        x = np.linspace(col_prop[col_prop['NAME']==col_name]['MINIMUM'],col_prop[col_prop['NAME']==col_name]['MAXIMUM'],100)
        mu = (col_prop[col_prop['NAME']==col_name]['MAXIMUM']-col_prop[col_prop['NAME']==col_name]['MINIMUM'])/2
        sigma = np.sqrt(col_prop[col_prop['NAME']==col_name]['VARIANCE'])
        
        y = ss.norm.cdf(x, mu, sigma)
        plt.plot(x, y, label='Normal Distribution CDF', color='#FF4500',linestyle='--')
        
    plt.title(col_name)
    plt.xlabel('Value')
    plt.ylabel('Cumulative Probability')
    plt.ylim(bottom=0)
    plt.xlim(left=0)
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Plot correlation matrix from Db2 Stats
def create_correlation_matrix(cont_col_list,table_name,ibm_db_conn):
    """Create a correlation matrix from a list of continuous features
    
    Input: cont_col_list - A list of features (e.g. ['AGE','FARE',...])
    
    Output: Plots a correlation matrix
    """

    corr_mtx = pd.DataFrame(columns=cont_col_list, index=cont_col_list)

    for combo in combinations_with_replacement(cont_cols, 2):

        col1=combo[0]
        col2=combo[1]


        sql = "SELECT CORRELATION("+col1+","+col2+") FROM "+table_name
        stmt = ibm_db.exec_immediate(ibm_db_conn, sql)
        row = ibm_db.fetch_assoc(stmt)


        to_append="{:.2f}".format(row['1'])
        corr_mtx.loc[col1,col2] = to_append
        corr_mtx.loc[col2,col1] = to_append
        
    corr_mtx=corr_mtx.astype(float)
    sns.heatmap(corr_mtx, annot=True, fmt = "0.2f",vmin=-1, vmax=1, center= 0, cmap= 'RdBu');

## 4. Data Exploration<a class="anchor" id="EDA"></a>

Without moving data from Db2, we will use built-in Db2 functions and stored procedures to perform basic statistical analysis of our entire dataset, and visualize the results to answer the following questions:
- Are there any missing values?
- What is the underlying distribution of numerical features?
- How many unique values for categorical features?
- Are any features correlated?

**Collect column statistics on the entire dataset**

In [ ]:
# Collect column statistics
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "CALL IDAX.COLUMN_PROPERTIES('intable=DATA.TITANIC , outtable=T_COL_PROP, "
sql+= "withstatistics=true, incolumn=PASSENGERID:id; SURVIVED:target')"
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql = "SELECT * FROM T_COL_PROP"
col_prop = pd.read_sql(sql,conn)

rc = ibm_db.close(ibm_db_conn)

In [ ]:
# Look at the column properties
col_prop.sort_values('COLNO')

**Observations**: Cardinality = Number of distinct values
- NAME, TICKET have too many distinct values to be useful. Can be dropped

In [ ]:
# Identify Columns with missing values
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "SELECT COLNO, NAME, TYPE,NUMMISSING,NUMMISSING+NUMINVALID+NUMVALID as NUMBER_OF_VALUES, "
sql+= "ROUND(dec(NUMMISSING,10,2)/(dec(NUMMISSING, 10,2)+dec(NUMINVALID, 10,2)+dec(NUMVALID, 10,2))*100,2) as PERCENT_NULL "
sql+= "from T_COL_PROP where NUMMISSING > 0 order by PERCENT_NULL DESC"
missing_vals = pd.read_sql(sql,conn)

rc = ibm_db.close(ibm_db_conn)

missing_vals

**Observations**:
- CABIN has 77% missing values
- AGE and EMBARKED require missing value imputation

**Plot CDFs of continuous features**

In [ ]:
# Call runstats on continuous columns
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "CALL sysproc.admin_cmd('runstats on table  DATA.TITANIC "
sql+= """with distribution on columns ("AGE","FARE") default num_quantiles 50');"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)


In [ ]:
# Select Quartiles from runstats results
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "select * from SYSSTAT.COLDIST where TABSCHEMA = 'DATA' and TABNAME = 'TITANIC' "
sql+= "and TYPE = 'Q'"
quartiles_df = pd.read_sql(sql,conn)

rc = ibm_db.close(ibm_db_conn)

# Convert values to numeric
quartiles_df.COLVALUE = pd.to_numeric(quartiles_df.COLVALUE)

In [ ]:
# Plot CDFs using quartile statistics
for col_name in ["AGE","FARE"]:
    plot_cdf_from_runstats_quartiles(col_name,quartiles_df, percentiles=True,norm_cdf=True)

**Observations:**
- AGE is well distributed, but takes a wide range of values. This feature should be zero-mean normalized. This CDF also confirms that 20% of its values are missing
- FARE is heavily skewed. Standardization/normalization will not help. This feature should be binned into a discrete categorical feature

**Countplots for categorical features**

In [ ]:
# Runstats on nominal columns
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "CALL sysproc.admin_cmd('runstats on table DATA.TITANIC "
sql+= """with distribution on columns ("SURVIVED","PCLASS","SEX","EMBARKED","SIBSP","PARCH") default num_freqvalues 10');"""

stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

In [ ]:
# Load runstats results
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "select * from SYSSTAT.COLDIST where TABSCHEMA = 'DATA' and TABNAME = 'TITANIC' "
sql+= "and TYPE = 'F'"
FREQ_VALUES = pd.read_sql(sql,conn)

rc = ibm_db.close(ibm_db_conn)

In [ ]:
# Countplot
_,ax = plt.subplots(3,2, figsize=(20,20))
nom_cols=["SURVIVED","PCLASS","SEX","EMBARKED","SIBSP","PARCH"]
count=0
for i in range(3):
    for j in range(2):
        COL_FREQ_VALUES = FREQ_VALUES.loc[(FREQ_VALUES.COLNAME == nom_cols[count]) & (FREQ_VALUES.VALCOUNT != -1)]
        sns.barplot(x='COLVALUE', y='VALCOUNT', data=COL_FREQ_VALUES,ax=ax[i,j])
        ax[i,j].set_title(nom_cols[count])
        ax[i,j].set(xlabel='Value')
        ax[i,j].set(ylabel='Count')
        count+=1

**Observations**:
- As expected:
    - most passengers did not survive.
    - there were more males than females
    - most passengers were in 3rd class

**Feature correlation matrix**

In [ ]:
# Correlation matrix
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

table_name = "DATA.TITANIC"
cont_cols = ["PASSENGERID", "SURVIVED", "PCLASS", "AGE","SIBSP","PARCH","FARE"]
create_correlation_matrix(cont_cols,table_name,ibm_db_conn)
    
rc = ibm_db.close(ibm_db_conn)

**Observations:**
- As expected, passenger class is negatively correlated with survival as many of the passengers who did not survive were in 3rd class.
- Age is negatively correlated as most of the survivors included young children

## 5. Splitting Data into Training, Validation, and Test Set<a class="anchor" id="Split-Data"></a>

We first create a view of the original data table, but without the features TICKET, CABIN, and NAME. This will simulate dropping these features without altering the raw data table.

We will then split the data into a training, validation, and testing set.

We create the following tables:

- T_TRAIN contains 64% of data - used for model training
- T_VAL contains 16% of data - used for model tuning (pruning)
- T_TEST contains 20% of data - used for model testing and evaluation

**Create a View. This simulates dropping columns from the raw data table**

In [ ]:
# Create view without TICKET, CABIN, and NAME features
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql= "CREATE VIEW TITANIC_VIEW AS SELECT PASSENGERID, SURVIVED, PCLASS, SEX, AGE, SIBSP, PARCH, FARE, EMBARKED FROM DATA.TITANIC"
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

**Train-test split**

In [ ]:
# Train-test split
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql= "CALL IDAX.SPLIT_DATA('intable=TITANIC_VIEW, id=PASSENGERID, traintable=T_TRAIN_FULL, testtable=T_TEST, fraction=0.8, seed=1')"
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

**Train-val split**

In [ ]:
# Train-val split
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql= """CALL IDAX.SPLIT_DATA('intable=T_TRAIN_FULL, 
        id=PASSENGERID, traintable=T_TRAIN, testtable=T_VAL, 
        fraction=0.8, seed=1')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

# Drop T_TRAIN_FULL as it is no longer needed
sql= """DROP TABLE T_TRAIN_FULL"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

## 6. Data Transformation<a class="anchor" id="Data-Transformation"></a>

**Available Data Transformations:**

- Impute Data (median, mean, most frequent, replace)
- Column Standardization: (x-mean)/stddev [-inf,+inf]
- Column Normalization: min-max scaling [0,1]
- Column Discretization

**Data Transformation Steps:**
1. Drop CABIN, TICKET, NAME features. These features are not relevant in predicting passenger survival. Done by creating a view in section 5
2. Data imputation - [Age] = mean, [EMARKED] ='S' --> most frequent value
3. Standardize [AGE]
4. Discretize [FARE] into bins of equal frequency

**Collect statistics on training set, to be used for transforming validation and test set**

In [ ]:
# Create T_STATS table that contains training dataset feature stats (mean, stdev, freq, etc) used for data 
# transformation in T_VAL, T_TEST
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "CALL IDAX.SUMMARY1000('intable=T_TRAIN,outtable=T_TRAIN_STATS')"
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

**Missing Value Imputation**

In [ ]:
# Missing values imputation for T_TRAIN
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "CALL IDAX.IMPUTE_DATA('intable=T_TRAIN,method=mean,inColumn=AGE')"
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql = "CALL IDAX.IMPUTE_DATA('intable=T_TRAIN,method=replace,nominalValue=S,inColumn=EMBARKED')"
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

In [ ]:
# Missing values imputation for T_VAL, T_TEST
ibm_db_conn, conn = connect_to_schema(schema,conn_str)


sql = """UPDATE T_VAL
         SET AGE = (SELECT AVERAGE FROM T_TRAIN_STATS_NUM WHERE COLUMNNAME='AGE')
         WHERE AGE IS NULL"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql = """UPDATE T_VAL
         SET EMBARKED = (SELECT MOSTFREQUENTVALUE FROM T_TRAIN_STATS_CHAR WHERE COLNAME='EMBARKED')
         WHERE EMBARKED IS NULL"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql = """UPDATE T_TEST
         SET AGE = (SELECT AVERAGE FROM T_TRAIN_STATS_NUM WHERE COLUMNNAME='AGE')
         WHERE AGE IS NULL"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql = """UPDATE T_TEST
         SET EMBARKED = (SELECT MOSTFREQUENTVALUE FROM T_TRAIN_STATS_CHAR WHERE COLNAME='EMBARKED')
         WHERE EMBARKED IS NULL"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

**Feature Standardization**

In [ ]:
# Load in data for later comparison
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "SELECT AGE FROM T_TRAIN ORDER BY RAND() FETCH FIRST 200 ROWS ONLY"
train = pd.read_sql(sql,conn)

sql = "SELECT AGE FROM T_VAL ORDER BY RAND() FETCH FIRST 200 ROWS ONLY"
val = pd.read_sql(sql,conn)

sql = "SELECT AGE FROM T_TEST ORDER BY RAND() FETCH FIRST 200 ROWS ONLY"
test = pd.read_sql(sql,conn)

rc = ibm_db.close(ibm_db_conn)

In [ ]:
# Standardization for T_TRAIN, write to _NORMED

ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """CALL IDAX.STD_NORM('intable=T_TRAIN, id=PASSENGERID,
        inColumn=SURVIVED:L;PCLASS:L;SEX:L;SIBSP:L;PARCH:L;EMBARKED:L;AGE:S;FARE:L,
        outtable=T_TRAIN_NORMED')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

# Drop T_TRAIN
sql= """DROP TABLE T_TRAIN"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

In [ ]:
# AGE Standardization for T_VAL, T_TEST
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """UPDATE T_VAL
         SET AGE = ((CAST(AGE AS FLOAT) - (SELECT AVERAGE FROM T_TRAIN_STATS_NUM WHERE COLUMNNAME='AGE'))/(SELECT STDDEV FROM T_TRAIN_STATS_NUM WHERE COLUMNNAME='AGE'))"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)


sql = """UPDATE T_TEST
         SET AGE = ((CAST(AGE AS FLOAT) - (SELECT AVERAGE FROM T_TRAIN_STATS_NUM WHERE COLUMNNAME='AGE'))/(SELECT STDDEV FROM T_TRAIN_STATS_NUM WHERE COLUMNNAME='AGE'))"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

# Rename AGE to STD_AGE for later comparison with T_TRAIN

sql= """ALTER TABLE T_VAL RENAME COLUMN AGE TO STD_AGE"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql= """ALTER TABLE T_TEST RENAME COLUMN AGE TO STD_AGE"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

In [ ]:
# Verify standardization was applied properly
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "SELECT STD_AGE FROM T_TRAIN_NORMED ORDER BY RAND() FETCH FIRST 200 ROWS ONLY"
train_standardized = pd.read_sql(sql,conn)

sql = "SELECT STD_AGE FROM T_VAL ORDER BY RAND() FETCH FIRST 200 ROWS ONLY"
val_standardized = pd.read_sql(sql,conn)

sql = "SELECT STD_AGE FROM T_TEST ORDER BY RAND() FETCH FIRST 200 ROWS ONLY"
test_standardized = pd.read_sql(sql,conn)

rc = ibm_db.close(ibm_db_conn)

# Plot Age and Fare
_,ax = plt.subplots(3,2, figsize=(20,20))

train['AGE'].hist(ax=ax[0,0])
ax[0,0].set(xlabel='TRAIN AGE')
ax[0,0].set(ylabel='Count')

train_standardized['STD_AGE'].hist(ax=ax[0,1])
ax[0,1].set(xlabel='TRAIN STD_AGE')
ax[0,1].set(ylabel='Count')

val['AGE'].hist(ax=ax[1,0])
ax[1,0].set(xlabel='VAL AGE')
ax[1,0].set(ylabel='Count')

val_standardized['STD_AGE'].hist(ax=ax[1,1])
ax[1,1].set(xlabel='VAL STD_AGE')
ax[1,1].set(ylabel='Count')

test['AGE'].hist(ax=ax[2,0])
ax[2,0].set(xlabel='TEST AGE')
ax[2,0].set(ylabel='Count')

test_standardized['STD_AGE'].hist(ax=ax[2,1])
ax[2,1].set(xlabel='TEST STD_AGE')
ax[2,1].set(ylabel='Count');

**Observation:** The values of the AGE feature have been standardized to have zero mean, and now take on a smaller range of values.

**Feature Discretization**

In [ ]:
# Load in data for later comparison
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "SELECT FARE FROM T_TRAIN_NORMED ORDER BY RAND() FETCH FIRST 200 ROWS ONLY"
train = pd.read_sql(sql,conn)

sql = "SELECT FARE FROM T_VAL ORDER BY RAND() FETCH FIRST 200 ROWS ONLY"
val = pd.read_sql(sql,conn)

sql = "SELECT FARE FROM T_TEST ORDER BY RAND() FETCH FIRST 200 ROWS ONLY"
test = pd.read_sql(sql,conn)

rc = ibm_db.close(ibm_db_conn)

In [ ]:
# Create bins
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """CALL IDAX.EFDISC('intable=T_TRAIN_NORMED,
        inColumn=FARE,bins=5,
        outtable=T_BTABLE')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)


rc = ibm_db.close(ibm_db_conn)

In [ ]:
# Apply discretization
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """CALL IDAX.APPLY_DISC('intable=T_TRAIN_NORMED, btable=T_BTABLE,
        outtable=T_TRAIN_CLEANED')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql = """CALL IDAX.APPLY_DISC('intable=T_VAL, btable=T_BTABLE,
        outtable=T_VAL_CLEANED')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql = """CALL IDAX.APPLY_DISC('intable=T_TEST, btable=T_BTABLE,
        outtable=T_TEST_CLEANED')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

# Drop T_TRAIN_NORMED, T_VAL, T_TEST

sql= """DROP TABLE T_TRAIN_NORMED"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql= """DROP TABLE T_VAL"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql= """DROP TABLE T_TEST"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

In [ ]:
# Verify all transformations were applied properly
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = "SELECT FARE FROM T_TRAIN_CLEANED"
train_cleaned = pd.read_sql(sql,conn)

sql = "SELECT FARE FROM T_VAL_CLEANED"
val_cleaned = pd.read_sql(sql,conn)

sql = "SELECT FARE FROM T_TEST_CLEANED"
test_cleaned = pd.read_sql(sql,conn)

# Get labels for Fare Bins
sql = "SELECT * FROM T_BTABLE"
df=pd.read_sql(sql,conn)
L=df["BREAK"].tolist()
labels = ["%.2f" % member for member in L]

rc = ibm_db.close(ibm_db_conn)

# Plot Age and Fare
_,ax = plt.subplots(3,2, figsize=(20,20))

train['FARE'].hist(ax=ax[0,0])
ax[0,0].set(xlabel='TRAIN FARE')
ax[0,0].set(ylabel='Count')

sns.countplot(x='FARE', data=train_cleaned, ax=ax[0,1])
ax[0,1].set(xlabel='TRAIN FARE')
ax[0,1].set(ylabel='Count')
ax[0,1].set_xticklabels(['<'+labels[0],'['+labels[0]+','+labels[1]+']','['+labels[1]+','+labels[2]+']','['+labels[2]+','+labels[3]+']','>'+labels[3]])

val['FARE'].hist(ax=ax[1,0])
ax[1,0].set(xlabel='VAL FARE')
ax[1,0].set(ylabel='Count')

sns.countplot(x='FARE', data=val_cleaned, ax=ax[1,1])
ax[1,1].set(xlabel='VAL FARE')
ax[1,1].set(ylabel='Count')
ax[1,1].set_xticklabels(['<'+labels[0],'['+labels[0]+','+labels[1]+']','['+labels[1]+','+labels[2]+']','['+labels[2]+','+labels[3]+']','>'+labels[3]])

test['FARE'].hist(ax=ax[2,0])
ax[2,0].set(xlabel='TEST AGE')
ax[2,0].set(ylabel='Count')

sns.countplot(x='FARE', data=test_cleaned, ax=ax[2,1])
ax[2,1].set(xlabel='TEST FARE')
ax[2,1].set(ylabel='Count')
ax[2,1].set_xticklabels(['<'+labels[0],'['+labels[0]+','+labels[1]+']','['+labels[1]+','+labels[2]+']','['+labels[2]+','+labels[3]+']','>'+labels[3]]);

**Observation:** The FARE feature that used to be heavily skewed to the left is now binned into a categorical feature that takes on discrete values. 

## 7. Model Training<a class="anchor" id="Model-Training"></a>

We will build a Decision Tree Classifier to make predictions on our data. We will use only default values for building the model, and then prune the model in section 9. For each model (pruned and unpruned) we will evaluate the model's accuracy on both the training and test set and compare performance.

In [ ]:
# Train model
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """CALL IDAX.GROW_DECTREE('model=titanic_dt, intable=T_TRAIN_CLEANED, 
        id=PASSENGERID, target=SURVIVED, incolumn=SURVIVED:nom;PCLASS:nom')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

**Return Model Parameters**

In [ ]:
# Return Model Parameters
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """SELECT * FROM 
        TITANIC_DT_MODEL"""
dt_model = pd.read_sql(sql,conn)

sql = """SELECT * FROM 
        TITANIC_DT_COLUMNS"""
dt_cols = pd.read_sql(sql,conn)

rc = ibm_db.close(ibm_db_conn)
print('Model Hyperparameters')
display(dt_model)
print('Model Input Features and Feature Importance')
display(dt_cols)

## 8. Evaluate Unpruned Model on Training and Test Set<a class="anchor" id="Eval-UP"></a>

We evaluate the model on each set by computing a confusion matrix and calculating the following important metrics:
- Classification Accuracy
- Recall = TPR (True Positive Rate)
- Precision = PPV (Positive Predicitive Value)

**Predict on Training and Test Set**

In [ ]:
# Predict on Training and Test Set
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """CALL IDAX.PREDICT_DECTREE('model=titanic_dt, intable=T_TRAIN_CLEANED, 
        outtable=T_DT_UP_TRAIN_PREDICTIONS')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql = """CALL IDAX.PREDICT_DECTREE('model=titanic_dt, intable=T_TEST_CLEANED, 
        outtable=T_DT_UP_TEST_PREDICTIONS')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

**Evaluate Model Performance**

In [ ]:
# Create confusion matrices
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """CALL IDAX.CONFUSION_MATRIX('intable=T_TRAIN_CLEANED, id=PASSENGERID, target=SURVIVED, 
        resulttable=T_DT_UP_TRAIN_PREDICTIONS, matrixtable=T_DT_UP_TRAIN_CM');"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)


sql = """CALL IDAX.CONFUSION_MATRIX('intable=T_TEST_CLEANED, id=PASSENGERID, target=SURVIVED, 
        resulttable=T_DT_UP_TEST_PREDICTIONS, matrixtable=T_DT_UP_TEST_CM');"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

In [ ]:
# Print confusion matrices
print('Unpruned model on Training Set:')
plot_conf_mtx('T_DT_UP_TRAIN_CM')
plt.show()
print('Unpruned model on Test Set:')
plot_conf_mtx('T_DT_UP_TEST_CM')
plt.show()

In [ ]:
# Confusion Matrix Statistics for unpruned tree on training and test sets

print('Unpruned Model on Training Set:')
get_conf_mtx_stats('T_DT_UP_TRAIN_CM')
print('Unpruned Model on Test Set:')
get_conf_mtx_stats('T_DT_UP_TEST_CM')


**Observations:**
Unpruned model performance:

- Unpruned Training Accuracy: 84.4%
- Unpruned Test Accuracy: 76.4%

## 9. Hyperparameter Tuning<a class="anchor" id="Hyperparam-Tuning"></a>

Now we prune the decision tree using the .PRUNE_DECTREE() function. This function works by pruning the tree from the bottom up and comparing the improvement in classification accuracy. Classification accuracy is measured on the validation set. This process continues until the accuracy of the model no longer improves.

If pruning works as expected, we should see a performance improvement on the test set and a performance decrease on the training set.

In [ ]:
# Prune Tree
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """CALL IDAX.PRUNE_DECTREE('model=titanic_dt, valtable=T_VAL_CLEANED')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

In [ ]:
#Investigate pruned model parameters
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """SELECT * FROM 
        TITANIC_DT_MODEL"""
dt_model = pd.read_sql(sql,conn)

rc = ibm_db.close(ibm_db_conn)
dt_model

**Observations:** We have decreased the depth of the tree by 1 and removed 10 nodes in the pruning process

## 10. Evaluate Pruned Model on Training and Test Set<a class="anchor" id="Eval-P-Test"></a>

**Predict with Pruned Model on Training and Test Set**

In [ ]:
# Predict again on Training and Test Set
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """CALL IDAX.PREDICT_DECTREE('model=titanic_dt, intable=T_TRAIN_CLEANED, 
        outtable=T_DT_P_TRAIN_PREDICTIONS')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql = """CALL IDAX.PREDICT_DECTREE('model=titanic_dt, intable=T_TEST_CLEANED,
        outtable=T_DT_P_TEST_PREDICTIONS')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

rc = ibm_db.close(ibm_db_conn)

**Evaluate Pruned Model**

In [ ]:
# Create confusion matrix
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """CALL IDAX.CONFUSION_MATRIX('intable=T_TRAIN_CLEANED, id=PASSENGERID, target=SURVIVED, 
        resulttable=T_DT_P_TRAIN_PREDICTIONS, matrixtable=T_DT_P_TRAIN_CM');"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql = """CALL IDAX.CONFUSION_MATRIX('intable=T_TEST_CLEANED, id=PASSENGERID, target=SURVIVED, 
        resulttable=T_DT_P_TEST_PREDICTIONS, matrixtable=T_DT_P_TEST_CM');"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)


rc = ibm_db.close(ibm_db_conn)

In [ ]:
# Print confusion matrix
print('Unpruned model on Training Set:')
plot_conf_mtx('T_DT_P_TRAIN_CM')
plt.show()
print('Unpruned model on Test Set:')
plot_conf_mtx('T_DT_P_TEST_CM')
plt.show()

In [ ]:
# Confusion Matrix Statistics for pruned tree on training and test sets
print('Unpruned Model on Training Set:')
get_conf_mtx_stats('T_DT_P_TRAIN_CM')
print('Unpruned Model on Test Set:')
get_conf_mtx_stats('T_DT_P_TEST_CM')


**Observations:**
Pruned model performance:
- Pruned Training Acc: 83.9% (-0.5%)
- Pruned Test Acc: 78.4% (+2%)

We can be confident that this pruned model will perform better on new, unseen data.

## 11. Deployed Model<a class="anchor" id="Deployment"></a>

In [ ]:
# Predict on new data
ibm_db_conn, conn = connect_to_schema(schema,conn_str)

sql = """CALL IDAX.PREDICT_DECTREE('model=titanic_dt, intable=T_VAL_CLEANED, 
        outtable=T_PREDICTIONS')"""
stmt = ibm_db.exec_immediate(ibm_db_conn, sql)

sql="select * FROM T_PREDICTIONS;"
df = pd.read_sql(sql,conn)

rc = ibm_db.close(ibm_db_conn)

df.head(10)

## 12. Conclusion<a class="anchor" id="Conclusion"></a>

In this notebook we have demonstrated the use of SQL Stored Procedures in building an end-to-end machine learning pipeline (data exploration, data transformation, model building, model tuning, and model evaluation) that is executed entirely in-database.

|Model | Training Accuracy | Testing Accuracy| Precision (Test, Class 1) | Recall (Test, Class 1)|
| --- | --- |--- | --- | --- |
|Unpruned DTC | 84.4% | 76.4%|71.6%|67.6%|
|Pruned DTC | 83.9% | 78.4%|79.6% | 60.6%|


## 13. Cleaning up Model and Tables<a class="anchor" id="Cleaning-up"></a>

In [ ]:
def remove_outstanding_tables(conn_str, intable_name, schema, print_error=False):
    # Delete all outstanding Tables
    ibm_db_conn = ibm_db.connect(conn_str,"","")
    conn = ibm_db_dbi.Connection(ibm_db_conn) 
    # drop summary
    try:
            sql = "CALL IDAX.DROP_SUMMARY1000('intable= " + schema + ".T_TRAIN_STATS')"
            stmt = ibm_db.exec_immediate(ibm_db_conn, sql)  
    except:
            if print_error: print("DROP_SUMMARY1000 removal error, skiping removal")  
    #______________________________________________________________________________________________
    # drop view
    try:
            sql= "DROP VIEW " + schema + "." + intable_name + "_VIEW"
            stmt = ibm_db.exec_immediate(ibm_db_conn, sql) 
    except:
            if print_error: print(schema + "." + intable_name +"view removal error, skiping removal")
    #______________________________________________________________________________________________
    # drop trainded models
    try:
            sql= "CALL IDAX.DROP_MODEL('model= " + schema + ".titanic_dt')"
            stmt = ibm_db.exec_immediate(ibm_db_conn, sql)
    except:
            if print_error: print("DROP_MODEL error, skiping removal")  
    try:
            sql= "CALL IDAX.DROP_MODEL('model= " + schema + ".titanic_knn')"
            stmt = ibm_db.exec_immediate(ibm_db_conn, sql)
    except:
            if print_error: print("DROP_MODEL error, skiping removal")  
    try:
            sql= "CALL IDAX.DROP_MODEL('model= " + schema + ".titanic_nb')"
            stmt = ibm_db.exec_immediate(ibm_db_conn, sql)
    except:
            if print_error: print("DROP_MODEL error, skiping removal")  
    #______________________________________________________________________________________________
    # drop any other table created in the schema
    get_tables_sql = "select NAME from sysibm.systables where CREATOR = '" + schema +"'"
    created_tables = pd.read_sql(get_tables_sql,conn).values.tolist()
    for table_list in created_tables:
        for table in table_list:
            sql= "DROP TABLE " + schema + "." + table
            try:
                stmt = ibm_db.exec_immediate(ibm_db_conn, sql)
            except:
                if print_error: print(table +" removal error, skiping removal")
    rc = ibm_db.close(ibm_db_conn)

In [ ]:
remove_outstanding_tables(conn_str, "TITANIC", "CLASS", print_error=True)

In [ ]:
# Check all tables properly deleted
ibm_db_conn = ibm_db.connect(conn_str,"","")
conn = ibm_db_dbi.Connection(ibm_db_conn)
sql="select NAME, CREATOR from sysibm.systables where CREATOR = 'CLASS';"
df = pd.read_sql(sql,conn)

rc = ibm_db.close(ibm_db_conn)
print('Connection Closed:',rc)

df